# DITL with ADCS physics

This notebook simulates multiple slews driven by the ACS and plots reaction-wheel stored momentum over time.


In [ ]:
# ruff: noqa: E402
from datetime import datetime, timedelta
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from rust_ephem import TLEEphemeris
from tqdm import tqdm

from conops import MissionConfig, Queue, QueueDITL
from conops.common import ACSMode
from conops.visualization import plot_ditl_telemetry
from conops.config.visualization import VisualizationConfig

# Shared ephemeris and targets
begin = datetime(2025, 1, 1)
end = begin + timedelta(hours=12)
ephem = TLEEphemeris(tle="../example.tle", begin=begin, end=end, step_size=10)

number_of_targets = 1000
np.random.seed(42)
target_ra = np.random.uniform(0, 360, number_of_targets)
target_dec = np.random.uniform(-90, 90, number_of_targets)
targids = list(range(10000, 10000 + number_of_targets))

def make_base_config(mtq_dipole: float) -> MissionConfig:
    cfg_local = MissionConfig.from_json_file("../example_config.json")
    acs = cfg_local.spacecraft_bus.attitude_control
    acs.spacecraft_moi = (45.0, 45.0, 45.0)
    acs.wheel_enabled = False
    acs.magnetorquers = [
        {"name": "mtq_x", "orientation": (1, 0, 0), "dipole_strength": mtq_dipole, "power_draw": 5.0},
        {"name": "mtq_y", "orientation": (0, 1, 0), "dipole_strength": mtq_dipole, "power_draw": 5.0},
        {"name": "mtq_z", "orientation": (0, 0, 1), "dipole_strength": mtq_dipole, "power_draw": 5.0},
    ]
    acs.magnetorquer_bfield_T = 3e-5
    acs.cp_offset_body = (0.25, 0.0, 0.0)
    acs.residual_magnetic_moment = (0.05, 0, 0)
    acs.drag_area_m2 = 1.3
    acs.drag_coeff = 2.2
    acs.solar_area_m2 = 1.3
    acs.solar_reflectivity = 1.3
    acs.use_msis_density = True
    cfg_local.constraint.ephem = ephem
    return cfg_local

def build_queue(cfg_local: MissionConfig) -> Queue:
    q = Queue(ephem=ephem, config=cfg_local)
    q.slew_distance_weight = 40.0
    for i in tqdm(range(number_of_targets), leave=False):
        q.add(
            merit=40,
            ra=target_ra[i],
            dec=target_dec[i],
            obsid=targids[i],
            name=f"pointing_{targids[i]}",
            exptime=1000,
            ss_min=300,
        )
    return q

def run_case(wheel_name: str, max_mom: float, max_torque: float, mtq_dipole: float) -> tuple[QueueDITL, dict]:
    cfg_case = make_base_config(mtq_dipole)
    acs_case = cfg_case.spacecraft_bus.attitude_control

    s = math.sqrt(2 / 3)
    c = math.sqrt(1 / 3)
    acs_case.wheels = [
        {"name": "rw1", "orientation": (+s, 0.0, +c), "max_torque": max_torque, "max_momentum": max_mom},
        {"name": "rw2", "orientation": (-s, 0.0, +c), "max_torque": max_torque, "max_momentum": max_mom},
        {"name": "rw3", "orientation": (0.0, +s, -c), "max_torque": max_torque, "max_momentum": max_mom},
        {"name": "rw4", "orientation": (0.0, -s, -c), "max_torque": max_torque, "max_momentum": max_mom},
    ]

    queue_case = build_queue(cfg_case)
    ditl_case = QueueDITL(config=cfg_case, ephem=ephem, begin=begin, end=end, queue=queue_case)
    ditl_case.acs._wheel_mom_margin = 1.0  # disable margin for what-if runs
    ditl_case.step_size = 10
    ditl_case.calc()

    mode_arr = np.array(ditl_case.mode)
    science_frac = float(np.mean(mode_arr == ACSMode.SCIENCE) * 100)
    slew_frac = float(np.mean(mode_arr == ACSMode.SLEWING) * 100)
    desat_events = getattr(ditl_case.acs, "desat_events", 0)
    wm = np.array(getattr(ditl_case, "wheel_momentum_fraction", []))
    wm_raw = np.array(getattr(ditl_case, "wheel_momentum_fraction_raw", []))
    wt = np.array(getattr(ditl_case, "wheel_torque_fraction", []))
    per_wheel = getattr(ditl_case, "wheel_per_wheel_max_raw", {}) or {
        getattr(w, "name", f"rw{i}"): abs(w.current_momentum) / w.max_momentum
        for i, w in enumerate(ditl_case.acs.reaction_wheels)
    }
    raw_peak = max(per_wheel.values()) if per_wheel else 0.0

    summary = {
        "name": wheel_name,
        "mtq_dipole": mtq_dipole,
        "max_mom": max_mom,
        "max_torque": max_torque,
        "science_%": science_frac,
        "slew_%": slew_frac,
        "desat_events": desat_events,
        "max_mom_frac": float(wm.max()) if wm.size else 0.0,
        "max_mom_frac_raw": float(wm_raw.max()) if wm_raw.size else 0.0,
        "p95_mom_frac": float(np.quantile(wm, 0.95)) if wm.size else 0.0,
        "p95_mom_frac_raw": float(np.quantile(wm_raw, 0.95)) if wm_raw.size else 0.0,
        "raw_peak_per_wheel": raw_peak,
        "per_wheel_peaks": per_wheel,
        "max_torque_frac": float(wt.max()) if wt.size else 0.0,
    }
    return ditl_case, summary


In [ ]:
# Single MTQ + wheel configuration
single_wheel = ("1.0Nms_0.06Nm", 1.0, 0.06)
single_mtq = 32 # A·m^2

ditl_case, summary_single = run_case(*single_wheel, single_mtq)
print(summary_single)

fig, axes = plot_ditl_telemetry(
    ditl_case,
    figsize=(12, 8),
    config=VisualizationConfig(font_family="DejaVu Sans"),
)
fig.suptitle(f"Case: {summary_single['name']} mtq={single_mtq}")
plt.show()


In [ ]:
mode = np.array(ditl_case.mode)
mtq_on = np.array(getattr(ditl_case, "mtq_power", [])) > 0
modes_on = {m.name: int((mode==m).sum()) for m in ACSMode}
print("MTQ on steps:", mtq_on.sum(), "of", mtq_on.size)
print("MTQ on by mode:", {m.name: int(((mode==m) & mtq_on).sum()) for m in ACSMode})


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from conops.common import ACSMode

# MTQ power and modes
mtq_power = np.array(getattr(ditl_case, "mtq_power", []))
mode = np.array(ditl_case.mode)

if mtq_power.size:
    print("MTQ power (W) min/median/max:", mtq_power.min(), np.median(mtq_power), mtq_power.max())
else:
    print("No mtq_power samples recorded")

mtq_on = mtq_power > 0
print("MTQ on steps:", mtq_on.sum(), "of", mtq_on.size)
print("MTQ on during science?", np.any(mtq_on & (mode == ACSMode.SCIENCE)))

# Rate of change of max wheel momentum fraction with MTQ on vs off
t = np.array(ditl_case.utime)
dt = np.diff(t, prepend=t[0])
h_raw = np.array(getattr(ditl_case, "wheel_momentum_fraction_raw", []))
rate = np.divide(np.diff(h_raw, prepend=h_raw[0]), dt, out=np.zeros_like(dt), where=dt > 0)
print("Mean dh/dt when MTQ on: ", rate[mtq_on].mean() if mtq_on.any() else np.nan, "per sec")
print("Mean dh/dt when MTQ off:", rate[~mtq_on].mean() if (~mtq_on).any() else np.nan, "per sec")

# Plot momentum with MTQ-on shading and MTQ power on a twin axis
time_hours = (t - t[0]) / 3600.0
fig, ax1 = plt.subplots(figsize=(10, 4))
ax1.plot(time_hours, h_raw, label="Max wheel H frac (raw)")
if mtq_on.any():
    ax1.fill_between(
        time_hours,
        0,
        1,
        where=mtq_on,
        color="gray",
        alpha=0.2,
        transform=ax1.get_xaxis_transform(),
        label="MTQ on",
    )
ax1.set_xlabel("Time (hours)")
ax1.set_ylabel("Max wheel H fraction (raw)")
ax2 = ax1.twinx()
ax2.plot(time_hours, mtq_power, color="tab:orange", alpha=0.6, label="MTQ power (W)")
ax2.set_ylabel("MTQ power (W)")
fig.legend(loc="upper right")
ax2.set_xlim(0,0.5)
fig.tight_layout()
plt.show()


In [ ]:
import numpy as np

def sanity_report(ditl):
    dt = float(ditl.step_size)
    t_hr = np.arange(len(ditl.mode)) * dt / 3600.0
    mode = np.array(ditl.mode)

    dist = np.array(getattr(ditl, "disturbance_total", []), dtype=float)
    mtq = np.array(getattr(ditl, "mtq_torque_mag", []), dtype=float)
    hold = np.array(getattr(ditl, "hold_torque_actual_mag", []), dtype=float)
    wheel = np.array(getattr(ditl, "wheel_torque_actual_mag", []), dtype=float)
    H_frac = np.array(getattr(ditl, "wheel_momentum_fraction_raw", []), dtype=float)
    max_mom = float(ditl.acs.reaction_wheels[0].max_momentum) if ditl.acs.reaction_wheels else 1.0

    sci = (mode == ACSMode.SCIENCE)

    print("Step size (s):", dt)
    print("Disturbance torque median/99% (N·m):", np.median(dist), np.quantile(dist, 0.99))
    print("MTQ torque median/99% (N·m):", np.median(mtq), np.quantile(mtq, 0.99))
    print("Wheel torque median/99% (N·m):", np.median(wheel), np.quantile(wheel, 0.99))
    print("Hold torque median/99% (N·m):", np.median(hold[sci]), np.quantile(hold[sci], 0.99))

    # Check SCIENCE drift rate
    H = H_frac * max_mom
    dHdt = np.gradient(H, dt)
    print("Mean dH/dt in SCIENCE (Nms/s):", dHdt[sci].mean())

    # Saturation stats
    print("Max wheel momentum frac:", np.max(H_frac))
    print("Frac steps >0.9:", np.mean(H_frac > 0.9))

sanity_report(ditl_case)

# Step-size sensitivity (rerun with two different dt if you want)
# for step in [5, 10, 20]:
#     ditl_case.step_size = step
#     ditl_case.calc()
#     print(f"--- step {step}s ---")
#     sanity_report(ditl_case)


In [ ]:
import numpy as np

dt = float(ditl_case.step_size)
mode = np.array(ditl_case.mode)

wheel_torque = np.array(getattr(ditl_case, "wheel_torque_actual_mag", []), dtype=float)
hold_torque = np.array(getattr(ditl_case, "hold_torque_actual_mag", []), dtype=float)
dist = np.array(getattr(ditl_case, "disturbance_total", []), dtype=float)

is_pass = (mode == ACSMode.PASS)

print("PASS steps:", is_pass.sum(), "of", len(mode))
print("Wheel torque median/99% (PASS):", np.median(wheel_torque[is_pass]), np.quantile(wheel_torque[is_pass], 0.99))
print("Wheel torque median/99% (NON-PASS):", np.median(wheel_torque[~is_pass]), np.quantile(wheel_torque[~is_pass], 0.99))

# Impulse comparison (Nms)
pass_impulse = np.sum(wheel_torque[is_pass]) * dt
nonpass_impulse = np.sum(wheel_torque[~is_pass]) * dt
print("Wheel impulse during PASS (Nms):", pass_impulse)
print("Wheel impulse outside PASS (Nms):", nonpass_impulse)

# Optional: disturbance vs pass wheel torque
print("Disturbance median/99% (PASS):", np.median(dist[is_pass]), np.quantile(dist[is_pass], 0.99))


In [ ]:
import numpy as np
from conops.common import ACSMode

# --- Helpers ---
def great_circle_deg(ra0, dec0, ra1, dec1):
    r0 = np.deg2rad(ra0); d0 = np.deg2rad(dec0)
    r1 = np.deg2rad(ra1); d1 = np.deg2rad(dec1)
    cosc = np.sin(d0)*np.sin(d1) + np.cos(d0)*np.cos(d1)*np.cos(r1-r0)
    return np.rad2deg(np.arccos(np.clip(cosc, -1.0, 1.0)))

# --- Inputs ---
mode = np.array(ditl_case.mode)
is_pass = mode == ACSMode.PASS
dt = float(ditl_case.step_size)

ra = np.array(ditl_case.ra)
dec = np.array(ditl_case.dec)

# dtheta per step (deg)
dtheta = np.zeros_like(ra)
dtheta[1:] = great_circle_deg(ra[:-1], dec[:-1], ra[1:], dec[1:])

# Kinematic alpha estimate (triangular profile): a ≈ 2*s/t^2 (deg/s^2)
alpha_est = 2.0 * dtheta / (dt * dt)

# Expected torque magnitude from alpha (Nm)
I = np.array(ditl_case.config.spacecraft_bus.attitude_control.spacecraft_moi, dtype=float)
if I.shape == (3,):
    Ieff = float(np.mean(I))
else:
    Ieff = float(np.trace(I) / 3.0)
T_expected = np.deg2rad(alpha_est) * Ieff

# Actual wheel torque magnitude (Nm) from telemetry
T_actual = np.array(ditl_case.wheel_torque_actual_mag)

print("PASS steps:", int(is_pass.sum()))
print("dtheta deg median/99% (PASS):", np.median(dtheta[is_pass]), np.quantile(dtheta[is_pass], 0.99))
print("T_expected median/99% (PASS):", np.median(T_expected[is_pass]), np.quantile(T_expected[is_pass], 0.99))
print("T_actual median/99% (PASS):", np.median(T_actual[is_pass]), np.quantile(T_actual[is_pass], 0.99))

# Per-wheel momentum deltas during PASS
wheel_hist = getattr(ditl_case, "wheel_momentum_history", {})
if wheel_hist:
    per_wheel_dH = {}
    for name, hist in wheel_hist.items():
        h = np.array(hist, dtype=float)
        dh = np.zeros_like(h)
        dh[1:] = h[1:] - h[:-1]
        per_wheel_dH[name] = np.median(np.abs(dh[is_pass]))
    print("Median |ΔH| per step during PASS (Nms):", per_wheel_dH)
else:
    print("No per-wheel history found.")


In [ ]:
import numpy as np
from conops.common import ACSMode

mode = np.array(ditl_case.mode)
is_pass = mode == ACSMode.PASS

for name, tq in ditl_case.wheel_torque_history.items():
    tq = np.array(tq)
    if is_pass.sum() == 0:
        print(name, "no PASS samples")
        continue
    print(
        name,
        "PASS median/99%:",
        np.median(np.abs(tq[is_pass])),
        np.quantile(np.abs(tq[is_pass]), 0.99),
    )


In [ ]:
import numpy as np
from conops.common import ACSMode

def great_circle_deg(ra0, dec0, ra1, dec1):
    r0 = np.deg2rad(ra0); d0 = np.deg2rad(dec0)
    r1 = np.deg2rad(ra1); d1 = np.deg2rad(dec1)
    cosc = np.sin(d0)*np.sin(d1) + np.cos(d0)*np.cos(d1)*np.cos(r1-r0)
    return np.rad2deg(np.arccos(np.clip(cosc, -1.0, 1.0)))

mode = np.array(ditl_case.mode)
is_pass = mode == ACSMode.PASS
dt = float(ditl_case.step_size)

ra = np.array(ditl_case.ra)
dec = np.array(ditl_case.dec)

dtheta = np.zeros_like(ra)
dtheta[1:] = great_circle_deg(ra[:-1], dec[:-1], ra[1:], dec[1:])

rate = dtheta / dt  # deg/s

if is_pass.sum() == 0:
    print("No PASS samples.")
else:
    r_pass = rate[is_pass]
    print("PASS tracking rate (deg/s) median/99%/max:",
          np.median(r_pass), np.quantile(r_pass, 0.99), r_pass.max())


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from conops.common import ACSMode

def great_circle_deg(ra0, dec0, ra1, dec1):
    r0 = np.deg2rad(ra0); d0 = np.deg2rad(dec0)
    r1 = np.deg2rad(ra1); d1 = np.deg2rad(dec1)
    cosc = np.sin(d0)*np.sin(d1) + np.cos(d0)*np.cos(d1)*np.cos(r1-r0)
    return np.rad2deg(np.arccos(np.clip(cosc, -1.0, 1.0)))

mode = np.array(ditl_case.mode)
is_pass = mode == ACSMode.PASS
dt = float(ditl_case.step_size)

ra = np.array(ditl_case.ra)
dec = np.array(ditl_case.dec)

dtheta = np.zeros_like(ra)
dtheta[1:] = great_circle_deg(ra[:-1], dec[:-1], ra[1:], dec[1:])

rate = dtheta / dt  # deg/s
alpha = 2.0 * dtheta / (dt * dt)  # deg/s^2 (triangular approx)

I = np.array(ditl_case.config.spacecraft_bus.attitude_control.spacecraft_moi, dtype=float)
if I.shape == (3,):
    Ieff = float(np.mean(I))
else:
    Ieff = float(np.trace(I) / 3.0)
T_expected = np.deg2rad(alpha) * Ieff
T_actual = np.array(ditl_case.wheel_torque_actual_mag)

t = np.arange(len(rate)) * dt / 3600.0

fig, ax1 = plt.subplots(figsize=(10, 4))
ax1.plot(t[is_pass], rate[is_pass], label="PASS rate (deg/s)", color="tab:blue")
ax1.set_xlabel("Time (hours)")
ax1.set_ylabel("Rate (deg/s)", color="tab:blue")
ax1.tick_params(axis='y', labelcolor="tab:blue")

ax2 = ax1.twinx()
ax2.plot(t[is_pass], T_expected[is_pass], label="T_expected (Nm)", color="tab:orange")
ax2.plot(t[is_pass], T_actual[is_pass], label="T_actual (Nm)", color="tab:green")
ax2.set_ylabel("Torque (Nm)", color="tab:green")
ax2.tick_params(axis='y', labelcolor="tab:green")

lines = ax1.get_lines() + ax2.get_lines()
labels = [l.get_label() for l in lines]
ax1.legend(lines, labels, loc="upper right")
ax1.set_title("PASS tracking rate vs wheel torque")
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import rust_ephem
from conops.config.groundstation import GroundStationRegistry

# Pick a pass
passes = getattr(ditl_case.acs.passrequests, "passes", [])
if not passes:
    print("No passes scheduled.")
else:
    gspass = passes[0]
    cfg = ditl_case.config
    gs_registry = cfg.ground_stations or GroundStationRegistry.default()
    station = gs_registry.get(gspass.station)

    # Build ground station ephem aligned to spacecraft ephem timestamps
    gs_ephem = rust_ephem.GroundEphemeris(
        latitude=station.latitude_deg,
        longitude=station.longitude_deg,
        height=station.elevation_m,
        begin=ephem.timestamp[0],
        end=ephem.timestamp[-1],
        step_size=ephem.step_size,
    )

    # Indices for pass timestamps
    timestamp_unix = np.array([dt.timestamp() for dt in ephem.timestamp])
    pass_times = np.array(gspass.utime)
    idx = np.searchsorted(timestamp_unix, pass_times)

    # Satellite and GS positions
    sat_pos = ephem.gcrs_pv.position[idx]
    gs_pos = gs_ephem.gcrs_pv.position[idx]

    # Elevation angle
    gs_to_sat = sat_pos - gs_pos
    gs_to_sat_unit = gs_to_sat / np.linalg.norm(gs_to_sat, axis=1, keepdims=True)
    earth_to_gs_unit = gs_pos / np.linalg.norm(gs_pos, axis=1, keepdims=True)
    cos_angle = np.sum(earth_to_gs_unit * gs_to_sat_unit, axis=1)
    elev = np.degrees(np.arcsin(np.clip(cos_angle, -1.0, 1.0)))

    # Tracking rate (deg/s) from pass RA/Dec
    ra = np.array(gspass.ra)
    dec = np.array(gspass.dec)
    dt = float(ditl_case.step_size)

    def great_circle_deg(ra0, dec0, ra1, dec1):
        r0 = np.deg2rad(ra0); d0 = np.deg2rad(dec0)
        r1 = np.deg2rad(ra1); d1 = np.deg2rad(dec1)
        cosc = np.sin(d0)*np.sin(d1) + np.cos(d0)*np.cos(d1)*np.cos(r1-r0)
        return np.rad2deg(np.arccos(np.clip(cosc, -1.0, 1.0)))

    dtheta = np.zeros_like(ra)
    dtheta[1:] = great_circle_deg(ra[:-1], dec[:-1], ra[1:], dec[1:])
    rate = dtheta / dt  # deg/s

    # Plot
    plt.figure(figsize=(6, 4))
    plt.scatter(elev, rate, s=20, alpha=0.8)
    plt.xlabel("Elevation angle (deg)")
    plt.ylabel("Tracking rate (deg/s)")
    plt.title(f"PASS tracking rate vs elevation ({gspass.station})")
    plt.grid(True, alpha=0.3)
    plt.show()


In [ ]:
import numpy as np

# Use the pass-only arrays you already built
rate_pass = rate  # deg/s, from the pass-only RA/Dec
pass_times = np.array(gspass.utime)

peak_idx = int(np.argmax(rate_pass))
peak_rate = rate_pass[peak_idx]
peak_time = pass_times[peak_idx]

print("Peak PASS rate (deg/s):", peak_rate)
print("Peak PASS time (unix):", peak_time)

# Map to nearest DITL timestep
ut = np.array(ditl_case.utime)
i = int(np.argmin(np.abs(ut - peak_time)))

# Per-wheel momentum at that time
for name, hist in ditl_case.wheel_momentum_history.items():
    h = np.array(hist, dtype=float)
    if i < len(h):
        print(f"{name} momentum (Nms) at peak:", h[i])

# Also show max wheel momentum fraction at that time
wm_raw = np.array(ditl_case.wheel_momentum_fraction_raw)
if i < len(wm_raw):
    print("Max wheel momentum fraction (raw) at peak:", wm_raw[i])


In [ ]:
import numpy as np

# pick the peak time you already found
peak_time = 1735718600.0
ut = np.array(ditl_case.utime)

# index of peak
i_peak = int(np.argmin(np.abs(ut - peak_time)))

# define a short baseline window just before the pass (e.g., 2–3 min)
dt = float(ditl_case.step_size)
pre = slice(max(0, i_peak - int(180/dt)), max(0, i_peak - int(60/dt)))

# net wheel H vector over time
s = (2/3) ** 0.5
c = (1/3) ** 0.5
axes = {
    "rw1": np.array([+s, 0.0, +c]),
    "rw2": np.array([-s, 0.0, +c]),
    "rw3": np.array([0.0, +s, -c]),
    "rw4": np.array([0.0, -s, -c]),
}

Hnet = []
for k in range(len(ut)):
    H = np.zeros(3)
    for name, hist in ditl_case.wheel_momentum_history.items():
        H += axes[name] * hist[k]
    Hnet.append(H)
Hnet = np.array(Hnet)

# bias‑removed net H at peak
H_bias = Hnet[pre].mean(axis=0)
H_peak = Hnet[i_peak]
H_delta = H_peak - H_bias

print("Raw |H| at peak:", np.linalg.norm(H_peak))
print("Bias‑removed |H| at peak:", np.linalg.norm(H_delta))


In [ ]:
# Vector drift check
H_net_vec = H_net  # from earlier
dHvec_dt = np.gradient(H_net_vec, ditl_case.step_size, axis=0)
mean_dHvec = dHvec_dt[science_mask].mean(axis=0)

# Disturbance torque vector (we don't store vector right now)
# so for now this is only a partial check.
print("Mean dH_net vector (SCIENCE):", mean_dHvec)


In [ ]:
mode = np.array(ditl_case.mode)
science = mode == ACSMode.SCIENCE

H_net = H_net
dHdt = (H_net[1:] - H_net[:-1]) / float(ditl_case.step_size)
valid = science[1:] & science[:-1]

mean_dHdt = dHdt[valid].mean(axis=0)

dist_vec = np.array(ditl_case.disturbance_vec, dtype=float)
mean_Tdist = dist_vec[1:][valid].mean(axis=0)

print("Mean dH_net/dt (SCIENCE-only diffs):", mean_dHdt)
print("Mean -T_dist (SCIENCE-only diffs):   ", -mean_Tdist)


In [ ]:
mode = np.array(ditl_case.mode)
mtq_power = np.array(getattr(ditl_case, "mtq_power", []), dtype=float)
mtq_on = mtq_power > 0
wm_raw = np.array(getattr(ditl_case, "wheel_momentum_fraction_raw", []), dtype=float)

if wm_raw.size == 0:
    print("No wheel_momentum_fraction_raw recorded."); 
else:
    dh = np.diff(wm_raw, prepend=wm_raw[0])
    dt = float(ditl_case.step_size)
    dhdt = dh / dt

    def stats(mask, label):
        mask = np.asarray(mask, dtype=bool)
        if mask.size == 0 or not mask.any():
            print(f"{label}: no samples")
            return
        sel = dhdt[mask]
        signed = np.sign(wm_raw[mask]) * sel
        print(f"{label}: n={sel.size}, mean dh/dt={sel.mean():.3e}, median={np.median(sel):.3e}")
        print(f"  signed dh/dt (unload<0): mean={signed.mean():.3e}, frac<0={(signed<0).mean():.2f}")

    stats(mtq_on & (mode == ACSMode.SLEWING), "MTQ on, slewing")
    stats(~mtq_on & (mode == ACSMode.SLEWING), "MTQ off, slewing")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

hours = (np.array(ditl_case.utime) - ditl_case.utime[0]) / 3600.0
mtq_proj = np.array(getattr(ditl_case, "mtq_proj_max", []), dtype=float)          # max |m×B| on any wheel (Nm)
mtq_mag  = np.array(getattr(ditl_case, "mtq_torque_mag", []), dtype=float)         # |m×B| norm (Nm)
wheel_t  = np.array(getattr(ditl_case, "wheel_torque_actual_mag", []), dtype=float) # |T_actual| from wheels (Nm)
mtq_on   = np.array(getattr(ditl_case, "mtq_power", []), dtype=float) > 0

fig, ax = plt.subplots(figsize=(10, 4))
ax.plot(hours, wheel_t, label="|T_actual| (wheels)", color="C0")
ax.plot(hours, mtq_proj, label="max |m×B| on any wheel", color="C1")
ax.plot(hours, mtq_mag, label="|m×B| norm", color="C2", alpha=0.6, linestyle="--")

# Shade MTQ-on intervals
on_edges = np.diff(mtq_on.astype(int))
starts = np.where(on_edges == 1)[0]
stops  = np.where(on_edges == -1)[0]
if mtq_on.any() and mtq_on[0]:
    starts = np.r_[0, starts]
if mtq_on.any() and len(stops) < len(starts):
    stops = np.r_[stops, len(mtq_on) - 1]
for s, e in zip(starts, stops):
    ax.axvspan(hours[s], hours[e], color="gray", alpha=0.15)

ax.set_xlabel("Time (hours)")
ax.set_ylabel("Torque (N·m)")
ax.set_title("Wheel torque vs MTQ torque projection")
ax.legend(loc="upper right", ncol=2)
plt.yscale('log')
plt.show()


In [ ]:
import numpy as np
obs = np.array(ditl_case.obsid)
dt = float(ditl_case.step_size)
dwells = []
i = 0
while i < len(obs):
    if obs[i] <= 0:
        i += 1; continue
    j = i
    while j < len(obs) and obs[j] == obs[i]:
        j += 1
    dwell_sec = (j - i) * dt
    dwells.append((obs[i], dwell_sec))
    i = j

dwell_df = pd.DataFrame(dwells, columns=["obsid", "dwell_s"])
print("min/median/max dwell (s):", dwell_df.dwell_s.min(), dwell_df.dwell_s.median(), dwell_df.dwell_s.max())
print("Any dwell < ss_min (300s)?", (dwell_df.dwell_s < 300).any())


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

t_hours = (np.array(ditl_case.utime) - ditl_case.utime[0]) / 3600.0
total = np.array(ditl_case.disturbance_total, dtype=float)
gg = np.array(ditl_case.disturbance_gg, dtype=float)
drag = np.array(ditl_case.disturbance_drag, dtype=float)
srp = np.array(ditl_case.disturbance_srp, dtype=float)
mag = np.array(ditl_case.disturbance_mag, dtype=float)

fig, ax = plt.subplots(figsize=(10, 4))
ax.plot(t_hours, total, label="Total", color="C0")
ax.plot(t_hours, gg, label="Gravity gradient", color="C1", alpha=0.7)
ax.plot(t_hours, drag, label="Aero drag", color="C2", alpha=0.7)
ax.plot(t_hours, srp, label="Solar pressure", color="C3", alpha=0.7)
ax.plot(t_hours, mag, label="Residual magnetic", color="C4", alpha=0.7)
ax.set_yscale("log")
ax.set_xlabel("Time (hours)")
ax.set_ylabel("Torque magnitude (N·m)")
ax.set_title("External disturbance torques over time")
ax.legend(loc="upper right", ncol=2)
ax.grid(alpha=0.2)
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

dt = float(ditl_case.step_size)
t_hours = (np.array(ditl_case.utime) - ditl_case.utime[0]) / 3600.0

# Disturbance torque magnitudes
gg = np.array(ditl_case.disturbance_gg, dtype=float)
drag = np.array(ditl_case.disturbance_drag, dtype=float)
srp = np.array(ditl_case.disturbance_srp, dtype=float)
mag = np.array(ditl_case.disturbance_mag, dtype=float)
dist_total = np.array(ditl_case.disturbance_total, dtype=float)

# MTQ projection (best-case unload) and wheel momentum fraction
mtq_proj = np.array(getattr(ditl_case, "mtq_proj_max", []), dtype=float)
wm_frac = np.array(getattr(ditl_case, "wheel_momentum_fraction_raw", []), dtype=float)

# Cumulative impulse from disturbances and MTQ
cum_gg = np.cumsum(gg * dt)
cum_drag = np.cumsum(drag * dt)
cum_srp = np.cumsum(srp * dt)
cum_mag = np.cumsum(mag * dt)
cum_mtq = np.cumsum(mtq_proj * dt)  # optimistic: assumes perfect opposing direction

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 6), sharex=True)

# Momentum fraction over time
ax1.plot(t_hours, wm_frac, label="Max wheel momentum fraction (raw)", color="C0")
ax1.set_ylabel("Momentum fraction")
ax1.set_title("Wheel momentum vs external torques/MTQ unload")
ax1.grid(alpha=0.2)
ax1.legend(loc="lower right")

# Cumulative impulse comparison
ax2.plot(t_hours, cum_gg, label="GG impulse", color="C1")
ax2.plot(t_hours, cum_drag, label="Drag impulse", color="C2")
ax2.plot(t_hours, cum_srp, label="SRP impulse", color="C3")
ax2.plot(t_hours, cum_mag, label="Residual mag impulse", color="C4")
ax2.plot(t_hours, np.cumsum(dist_total * dt), label="Total disturbance impulse", color="C5", linestyle="--")
ax2.plot(t_hours, cum_mtq, label="MTQ projected impulse (best-case oppose)", color="C6")
ax2.set_xlabel("Time (hours)")
ax2.set_ylabel("Impulse (N·m·s)")
ax2.grid(alpha=0.2)
ax2.set_yscale("log")
ax2.set_ylim(bottom=1e-21)
ax2.legend(loc="lower right", ncol=2)

plt.tight_layout()
plt.show()


In [ ]:
import numpy as np

dt = float(ditl_case.step_size)
t_hr = np.arange(len(ditl_case.mode)) * dt / 3600.0

H_frac = np.array(ditl_case.wheel_momentum_fraction_raw, dtype=float)
T_dist = np.array(getattr(ditl_case, "disturbance_total", []), dtype=float)
T_wheel = np.array(getattr(ditl_case, "wheel_torque_actual_mag", []), dtype=float)
mtq_p = np.array(getattr(ditl_case, "mtq_power", []), dtype=float)
mode = np.array(ditl_case.mode)

# pick a plateau window manually
t0, t1 = 0.15,0.35  # hours
mask = (t_hr >= t0) & (t_hr <= t1)

print("Plateau window:", t0, "to", t1, "hrs")
print("mean dH/dt:", np.mean(np.gradient(H_frac[mask], dt)))
print("mean |T_wheel|:", np.mean(T_wheel[mask]))
print("mean |T_dist|:", np.mean(T_dist[mask]))
print("MTQ on?", np.any(mtq_p[mask] > 0))
print("SCIENCE only?", np.all(mode[mask] == ACSMode.SCIENCE))


In [ ]:
import numpy as np

dt = float(ditl_case.step_size)
t_hr = np.arange(len(ditl_case.mode)) * dt / 3600.0
mode = np.array(ditl_case.mode)

T_dist = np.array(getattr(ditl_case, "disturbance_total", []), dtype=float)
T_wheel = np.array(getattr(ditl_case, "wheel_torque_actual_mag", []), dtype=float)
H_frac = np.array(ditl_case.wheel_momentum_fraction_raw, dtype=float)

t0, t1 = 0.15, 0.35
mask = (t_hr >= t0) & (t_hr <= t1)

print("any slew in window?", np.any(mode[mask] == ACSMode.SLEWING))
print("mean |T_dist|:", T_dist[mask].mean())
print("mean |T_wheel|:", T_wheel[mask].mean())
print("mean dH/dt:", np.gradient(H_frac[mask], dt).mean())

# Dump a few samples to see if wheel torque is ever nonzero
for idx in np.where(mask)[0][:5]:
    print(f"t={t_hr[idx]:.3f}h  mode={ACSMode(mode[idx]).name}  T_dist={T_dist[idx]:.3e}  T_wheel={T_wheel[idx]:.3e}")


In [ ]:
t0, t1 = 0.15, 0.35
mask = (t_hr >= t0) & (t_hr <= t1)

hold_target = np.array(ditl_case.hold_torque_target_mag, dtype=float)
hold_actual = np.array(ditl_case.hold_torque_actual_mag, dtype=float)
dist = np.array(ditl_case.disturbance_total, dtype=float)

print("mean |T_hold_target|:", hold_target[mask].mean())
print("mean |T_hold_actual|:", hold_actual[mask].mean())
print("mean |T_dist|:", dist[mask].mean())


In [ ]:
# Expected bias from integral of disturbance torque in a pre-plateau build-up window
t0, t1 = 0.05, 0.15  # adjust to the ramp-up before the plateau
mask = (t_hr >= t0) & (t_hr <= t1)
expected_bias = np.trapezoid(dist[mask], t_hr[mask] * 3600.0)  # Nms
print("Expected bias (Nms):", expected_bias)


In [ ]:
max_mom = float(ditl_case.acs.reaction_wheels[0].max_momentum)
plateau_mask = (t_hr >= 0.15) & (t_hr <= 0.35)
plateau_bias = np.mean(H_frac[plateau_mask]) * max_mom
print("Plateau bias (Nms):", plateau_bias)
print("Ratio plateau/expected:", plateau_bias / expected_bias)


In [ ]:
dt = float(ditl_case.step_size)
H_frac = np.array(ditl_case.wheel_momentum_fraction_raw, dtype=float)
max_mom = float(ditl_case.acs.reaction_wheels[0].max_momentum)
H_max = H_frac * max_mom

# pick the ramp-up window where H is growing
t0, t1 = 0.05, 0.15
mask = (t_hr >= t0) & (t_hr <= t1)

dHdt = np.gradient(H_max, dt)

delta_H = H_max[mask][-1] - H_max[mask][0]
integral_dH = np.trapezoid(dHdt[mask], t_hr[mask] * 3600.0)

print("ΔH_max (Nms):", delta_H)
print("∫ dH/dt (Nms):", integral_dH)


In [ ]:
dt = float(ditl_case.step_size)
mode = np.array(ditl_case.mode)
mtq = np.array(getattr(ditl_case, "mtq_power", []), dtype=float)
H_frac = np.array(ditl_case.wheel_momentum_fraction_raw, dtype=float)
max_mom = float(ditl_case.acs.reaction_wheels[0].max_momentum)
H_max = H_frac * max_mom
dHdt = np.gradient(H_max, dt)

science_mask = (mode == ACSMode.SCIENCE) & (mtq <= 0.0)
print("Mean dH/dt in SCIENCE (MTQ off):", dHdt[science_mask].mean())
print("Frac of SCIENCE samples with |dH/dt|>0:", np.mean(np.abs(dHdt[science_mask]) > 0))


In [ ]:
wheel_grid = [
    ("1.0Nms_0.06Nm", 1.0, 0.06),
    ("2.0Nms_0.12Nm", 2.0, 0.12),
    ("1.2Nms_0.25Nm", 1.2, 0.25),
    ("4.0Nms_0.25Nm", 4.0, 0.25),
    ("2.5Nms_0.50Nm", 2.5, 0.50),
    ("8.0Nms_0.25Nm", 8.0, 0.25),
    ("5.0Nms_0.50Nm", 5.0, 0.50),
]
mtq_grid = [32, 50, 100]

results = []
ditls = {}
for mtq in mtq_grid:
    for name, mom, torq in wheel_grid:
        key = f"{name}_mtq{mtq}"
        ditl_case, summary = run_case(name, mom, torq, mtq)
        ditls[key] = ditl_case
        summary["key"] = key
        results.append(summary)

df = pd.DataFrame(results).set_index("key")
df_sorted = df.sort_values(
    by=["science_%", "slew_%", "desat_events", "max_mom_frac", "max_torque_frac"],
    ascending=[False, True, True, True, True],
)
display(df_sorted.head())

# Plot the top-ranked case
case_to_plot = df_sorted.index[0]
fig, axes = plot_ditl_telemetry(
    ditls[case_to_plot],
    figsize=(12, 8),
    config=VisualizationConfig(font_family="DejaVu Sans"),
)
fig.suptitle(f"Case: {case_to_plot}")
plt.show()


In [ ]:
import numpy as np
from collections import Counter
from conops.common import ACSMode, ACSCommandType

# Timebase
t = np.array(ditl.utime)
dt = np.diff(t, prepend=t[0])
hrs = dt / 3600

m = np.array(ditl.mode)
time_by_mode = {
    "SCIENCE": np.sum(hrs[m == ACSMode.SCIENCE]),
    "SLEWING": np.sum(hrs[m == ACSMode.SLEWING]),
    "PASS": np.sum(hrs[m == ACSMode.PASS]),
    "SAFE": np.sum(hrs[m == ACSMode.SAFE]),
    "CHARGING": np.sum(hrs[m == ACSMode.CHARGING]),
    "SAA": np.sum(hrs[m == ACSMode.SAA]),
}
print("Hours by mode:", time_by_mode)

# Desat
desat_steps = getattr(ditl, "desat_time_steps", 0)
print(f"Desat hours: {desat_steps * ditl.step_size / 3600:.2f}")

# Wheel margins
wm = np.array(getattr(ditl, "wheel_momentum_fraction", []))
wt = np.array(getattr(ditl, "wheel_torque_fraction", []))
if wm.size:
    print(f"Wheel momentum: max={wm.max():.2f}, p95={np.quantile(wm,0.95):.2f}")
if wt.size:
    print(f"Wheel torque: max={wt.max():.2f}, p95={np.quantile(wt,0.95):.2f}")

# Slew outcomes
slews = [
    c for c in ditl.acs.executed_commands
    if c.command_type == ACSCommandType.SLEW_TO_TARGET and c.slew
]
dists = [c.slew.slewdist for c in slews]
rejected = getattr(ditl.acs, "headroom_rejects", 0)
print(f"Slews: {len(slews)} executed, {rejected} rejected; dist mean={np.mean(dists):.1f} deg")

# Simple histograms (numpy bins)
hist_mom, bins_m = np.histogram(wm, bins=np.linspace(0,1,11)) if wm.size else ([],[])


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

time_hours = (np.array(ditl.utime) - ditl.utime[0]) / 3600

plt.figure(figsize=(10,5))
#plt.plot(time_hours, ditl.disturbance_total, label="Total")
plt.plot(time_hours, ditl.disturbance_gg, label="Gravity gradient")
plt.plot(time_hours, ditl.disturbance_drag, label="Aero drag")
plt.plot(time_hours, ditl.disturbance_srp, label="Solar pressure")
plt.plot(time_hours, ditl.disturbance_mag, label="Magnetic (residual)")
plt.xlabel("Time (hours)")
plt.ylabel("Torque magnitude (N·m)")
plt.legend()
plt.yscale("log")
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
from datetime import datetime, timezone

# enable MSIS
cfg.spacecraft_bus.attitude_control.use_msis_density = True

# after your DITL calc:
acs = ditl.acs
ephem = ditl.ephem

densities = []
alts_km = []
for ut in ditl.utime:
    idx = ephem.index(datetime.fromtimestamp(ut, tz=timezone.utc))
    r_vec = np.array(ephem.gcrs_pv.position[idx])
    # normalize to meters if in km
    if np.linalg.norm(r_vec) < 1e6:
        r_vec = r_vec * 1000.0
    alt_m = max(0.0, np.linalg.norm(r_vec) - 6378e3)
    lat = float(getattr(ephem, "lat", [0])[idx]) if hasattr(ephem, "lat") else 0.0
    lon = float(getattr(ephem, "long", [0])[idx]) if hasattr(ephem, "long") else 0.0
    rho = acs._atmospheric_density(ut, alt_m, lat, lon)
    densities.append(rho)
    alts_km.append(alt_m / 1000.0)

# Plot density vs time
import matplotlib.pyplot as plt
time_hours = (np.array(ditl.utime) - ditl.utime[0]) / 3600
plt.semilogy(time_hours, densities)
plt.xlabel("Time (hours)")
plt.ylabel("Density (kg/m^3)")
plt.tight_layout()
plt.show()


In [ ]:
acs = ditl.acs
acs._use_msis = True  # ensure MSIS path
ut0 = ditl.utime[0]
idx = ditl.ephem.index(datetime.fromtimestamp(ut0, tz=timezone.utc))
r = np.array(ditl.ephem.gcrs_pv.position[idx])
if np.linalg.norm(r) < 1e6: r = r*1000
alt = np.linalg.norm(r) - 6378e3
lat = float(getattr(ditl.ephem, "lat", [0])[idx])
lon = float(getattr(ditl.ephem, "long", [0])[idx])
rho = acs._atmospheric_density(ut0, alt, lat, lon)
print(f"alt={alt/1000:.1f} km rho={rho:.3e} kg/m^3")


In [ ]:
alt

In [ ]:
ditl.print_statistics()

In [ ]:
from conops.common import ACSCommandType, ACSMode
time_h = (np.array(ditl.utime) - ditl.utime[0]) / 3600
slew_cmds = [c for c in ditl.acs.executed_commands if c.command_type == ACSCommandType.SLEW_TO_TARGET]
desat_cmds = [c for c in ditl.acs.executed_commands if c.command_type == ACSCommandType.DESAT]
print("DESAT windows:", [(c.execution_time, getattr(c, "duration", None)) for c in desat_cmds])

# Overlay where mode==SLEWING to see it spans settle/desat
import matplotlib.pyplot as plt
m = np.array(ditl.mode)
plt.plot(time_h, ditl.ra)
plt.scatter(time_h[m == ACSMode.SLEWING], np.array(ditl.ra)[m == ACSMode.SLEWING], s=8, color="r")


In [ ]:
from conops.common import ACSCommandType

cmds = [c for c in ditl.acs.executed_commands if c.command_type == ACSCommandType.SLEW_TO_TARGET]
for c in cmds:
    dist = c.slew.slewdist          # deg along great circle
    t = c.slew.slewtime             # s (includes settle)
    avg_rate = dist / t if t else 0
    a = getattr(c.slew, "_accel_override", None)
    vmax = getattr(c.slew, "_vmax_override", None)
    print(f"obsid={c.slew.obsid} dist={dist:.2f}deg time={t:.1f}s avg={avg_rate:.3f} deg/s accel={a} vmax={vmax}")


In [ ]:
import numpy as np
from conops.common import separation

# great-circle step-to-step rate
rates = []
for i in range(1, len(ditl.ra)):
    r = separation(
        [np.deg2rad(ditl.ra[i-1]), np.deg2rad(ditl.dec[i-1])],
        [np.deg2rad(ditl.ra[i]),   np.deg2rad(ditl.dec[i])]
    ) # deg
    d = np.rad2deg(r)
    rates.append(d / ditl.step_size)  # deg/s
print("Max step rate:", max(rates), "deg/s")


In [ ]:
import numpy as np
from conops.common import ACSCommandType
dists = [c.slew.slewdist for c in ditl.acs.executed_commands
         if c.command_type == ACSCommandType.SLEW_TO_TARGET and c.slew]
print("mean dist:", np.mean(dists), "deg", "max:", np.max(dists), "deg")


In [ ]:
import numpy as np
from conops.common import ACSCommandType, separation

# Slew distances actually flown
dists = [c.slew.slewdist for c in ditl.acs.executed_commands
         if c.command_type == ACSCommandType.SLEW_TO_TARGET and c.slew]
print(f"Count={len(dists)}, mean={np.mean(dists):.1f} deg, median={np.median(dists):.1f}, max={np.max(dists):.1f}")

# Time in SLEWING
mode_array = np.array(ditl.mode)
frac_slewing = np.mean(mode_array == 1) * 100
print(f"SLEWING fraction: {frac_slewing:.1f}%")

# Effective accel/rate per slew
for c in ditl.acs.executed_commands:
    if c.command_type == ACSCommandType.SLEW_TO_TARGET and c.slew:
        print(f"obsid={c.slew.obsid} dist={c.slew.slewdist:.1f} deg, "
              f"accel={getattr(c.slew,'_accel_override',None)}, "
              f"vmax={getattr(c.slew,'_vmax_override',None)}")


In [ ]:
import numpy as np
from conops.common import ACSMode

mode_arr = np.array(ditl.mode)
science_frac = np.mean(mode_arr == ACSMode.SCIENCE) * 100
slew_frac = np.mean(mode_arr == ACSMode.SLEWING) * 100
print(f"SCIENCE: {science_frac:.1f}%")
print(f"SLEWING: {slew_frac:.1f}%")
